In [3]:
#2023.2.13 I could work late playing python and still felt engross.
#2023.2.21 2.0 Start projects will display timing; Display function which can show study time of the day.

import tkinter as tk
from tkinter import simpledialog

def show_text_dialog(string):
    root = tk.Tk()
    
    root.withdraw()

    text = simpledialog.askstring("Input",string, parent=root)

    return text
    
    root.destroy()


In [4]:
import datetime

class Record:
    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.subjects = []
        self.emotions = []
        
    def get_start_time(self):
        self.start_time = datetime.datetime.now()
    
    def get_end_time(self):
        self.end_time = datetime.datetime.now()
    
    def save_to_datafile(self, filename):
        with open(filename, "a") as f: #用a模式打开文件不会清除txt原有内容
            f.write(f"Start time: {self.start_time}\n")
            for subject in self.subjects:
                f.write('Subject: ' + subject[0] + f" time:{subject[1]}\n")
            for emotion in self.emotions:
                f.write('Emotion: ' + emotion[0] + f" time:{emotion[1]}\n")
            f.write(f"End time: {self.end_time}\n\n")
    
    def from_datafile(self, filename, records):
        with open(filename, "r") as f:
            lines = f.readlines()
        for line in lines:
            if line[0:10] == 'Start time':
                record = Record()
                start_time_str = line.split(": ")[1].strip()
                start_time = datetime.datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S.%f")
                record.start_time = start_time
            if line[0:8] == 'End time':
                end_time_str = line.split(": ")[1].strip()
                end_time = datetime.datetime.strptime(end_time_str, "%Y-%m-%d %H:%M:%S.%f")
                record.end_time = end_time
                records.append(record) #将读到的record放到records队尾
                
    def emotion_det(self):
        text = show_text_dialog("How do you feel?") 
        self.emotions.append([text,datetime.datetime.now()])
        
    def subject_det(self):
        text = show_text_dialog("What do you want to do?")  
        self.subjects.append([text,datetime.datetime.now()])

    def subject_syn(self): #syncronize subjects
        pass


In [88]:
from datetime import date
def display(year,display_start_month,display_start_day,display_end_month,display_end_day):
    
    f_date = date(year, display_start_month, display_start_day)
    l_date = date(year, display_end_month, display_end_day)
    s_date = date(year, 1, 1)
    delta = l_date - f_date
    delta_yday = f_date - s_date
    delta_day = delta.days
    start_yday = delta_yday.days
    
    record = Record()
    records = []
    record.from_datafile('DataFile.txt',records)
    for i in range(0,len(records)):
        if i == 0:
            h_sub,m_sub,s_sub = time_sub(records[0].end_time.hour,records[0].end_time.minute,records[0].end_time.second,records[0].start_time.hour,records[0].start_time.minute,records[0].start_time.second)
            h_add = h_sub
            m_add = m_sub
            s_add = s_sub
            day_record = []
        else:
            if records[i].start_time.day == records[i-1].start_time.day:
                h_sub,m_sub,s_sub = time_sub(records[i].end_time.hour,records[i].end_time.minute,records[i].end_time.second,records[i].start_time.hour,records[i].start_time.minute,records[i].start_time.second)
                h_add,m_add,s_add = time_add(h_add,m_add,s_add,h_sub,m_sub,s_sub)

            if records[i].start_time.day != records[i-1].start_time.day:
                year = records[i-1].start_time.year
                month = records[i-1].start_time.month
                day = records[i-1].start_time.day
                day_record.append([year, month, day, round(h_add+m_add/60,1)])

                h_sub,m_sub,s_sub = time_sub(records[i].end_time.hour,records[i].end_time.minute,records[i].end_time.second,records[i].start_time.hour,records[i].start_time.minute,records[i].start_time.second)
                h_add = h_sub
                m_add = m_sub
                s_add = s_sub

            if i == len(records)-1:
                year = records[i].start_time.year
                month = records[i].start_time.month
                day = records[i].start_time.day
                day_record.append([year, month, day, round(h_add+m_add/60,1)])

    if(year%4)==0:
        if (year % 100) == 0:
            if (year % 400) == 0:
                feb = 29
            else:
                feb = 28
        else:
            feb = 29
    else:
        feb = 28

    mday = [31,feb,31,30,31,30,31,31,30,31,30,31]
    day_record_Ayear = []
    for month in range(12):
        for day in range(mday[month]):
            day_record_Ayear.append([year,month+1,day+1,0])

    j=0                               
    for j in range(len(day_record)):
        if day_record[j][0] == year:
            break

    for i in range(len(day_record_Ayear)):
        if day_record_Ayear[i][1] == day_record[j][1] and day_record_2023[i][2] == day_record[j][2]:
            day_record_Ayear[i][3] = day_record[j][3]
            j = j+1
            if j==len(day_record):
                break


    import matplotlib.pyplot as plt
    x = range(delta_day+1)
    y = [day_record_Ayear[start_yday + i][3] for i in x]

    plt.figure(figsize=(8, 6),dpi=80) #表的比例和大小
    plt.plot(x, y,color= 'r',linestyle='--', label='StudyRecord')

    x_label = ["{}.{}".format(day_record_Ayear[i+start_yday][1],day_record_Ayear[i+start_yday][2]) for i in x]
    plt.xticks(x[::1],x_label[::1]) #逐个显示
    plt. grid(linestyle='--', alpha=0.5, linewidth=2)

    plt.xlabel('Time/day')
    plt .ylabel( 'StudyTime/hour' )
    plt. title( 'StudyRecord' )
    plt.show()


In [30]:
def time_sub(end_hour,end_min,end_sec,sta_hour,sta_min,sta_sec):
    h_sub = end_hour-sta_hour
    m_sub = end_min-sta_min
    s_sub = end_sec-sta_sec
    if h_sub<0:
        h_sub = 24+h_sub
    if m_sub<0:
        m_sub = 60+m_sub
        h_sub = h_sub-1
    if s_sub<0:
        s_sub = 60+s_sub
        m_sub = m_sub-1
    return h_sub,m_sub,s_sub

def time_add(end_hour,end_min,end_sec,sta_hour,sta_min,sta_sec):
    h_add = end_hour+sta_hour
    m_add = end_min+sta_min
    s_add = end_sec+sta_sec
    if s_add>59:
        s_add = s_add-60
        m_add = m_add+1
    if m_add>59:
        m_add = m_add-60
        h_add = h_add+1
    return h_add,m_add,s_add


In [21]:
import cv2
import datetime
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW) #0表示默认camera
cap.open(0)
cap.set(3,320)
cap.set(4,240)


record = Record()
record.get_start_time()
project_start_time = None
while cap.isOpened():
    flag, frame = cap.read()
    
    if project_start_time != None:
        now = datetime.datetime.now()
        hour_sub,min_sub,sec_sub = time_sub(now.hour,now.minute,now.second,project_start_time.hour,project_start_time.minute,project_start_time.second)
        time_str = '{:.0f}:{:.0f}:{:.0f}'.format(hour_sub,min_sub,sec_sub)
        cv2.putText(frame,time_str,(10,40),cv2.FONT_HERSHEY_SIMPLEX,0.6, (255,0,0), 2)
    cv2.imshow('Window',frame)
    
    key_pressed = cv2.waitKey(60)
    if key_pressed == 32: #Spacebar
        record.emotion_det()
    
    if key_pressed == 9: #Tab
        record.subject_det()
        project_start_time = datetime.datetime.now()
    
    if key_pressed == 27: #ESC
        break
    
    
record.get_end_time()
cap.release()
cv2.destroyAllWindows()
record.save_to_datafile('DataFile.txt')
